[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Etiquetado_openai.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [40]:
!pip install -q PyPDF2 python-docx openpyxl python-pptx xlrd transformers office365-rest-python-client msal requests

In [60]:
import os
import io
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader
import docx
import openpyxl
from pptx import Presentation
import xlrd
from transformers import pipeline
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import os
import requests
import msal
import csv
import getpass
from openai import OpenAI

# Configuración

In [61]:
api_key = getpass.getpass("Introduce tu OpenAI API Key: ")
client = OpenAI(api_key=api_key)

Introduce tu OpenAI API Key: ··········


In [62]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"  # tu nuevo Client ID
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"

# Conección con OneDrive


In [63]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()

To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code FUXUZY9Z to authenticate.


#Funciones

Clasificador Hugging Face Zero-Shot


In [66]:
etiquetas = [
    "contrato", "arrendamiento", "vivienda", "modelo", "legal", "inmobiliaria",
    "admitidos", "cursos", "monográficos", "educación", "registro",
    "reserva", "inmueble", "alquiler", "NDA", "confidencialidad", "acuerdo", "plantilla",
    "unilateral", "UE", "OEPM", "presentación", "anexo", "personal", "informe",
    "análisis", "estados financieros", "contabilidad", "proyecto",
    "biblioteca", "datos", "csv", "inventario", "opción de compra", "freelancer",
    "formulario", "registro", "administrativo", "prueba", "test", "justificante",
    "bachillerato", "HH-CCSS", "temporada", "finca rústica", "rescisión",
    "empresa", "académico", "prórroga", "proyecto",
    "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019",
    "2020", "2021", "2022", "2023", "2024", "2025", "2026", "2027", "2028", "2029", "2030"
]

def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = texto.lower()
    texto = re.sub(r"[\r\n\t]+", " ", texto)
    texto = re.sub(r"[^a-záéíóúüñ0-9\s]", "", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

def clasificar_texto(texto: str, max_etiquetas: int = 5) -> list:
    """
    Clasifica un texto usando GPT-5-mini y devuelve hasta max_etiquetas etiquetas descriptivas.
    """
    texto_limpio = limpiar_texto(texto)

    prompt = f"""
    Lee el siguiente texto y sugiere hasta {max_etiquetas} etiquetas cortas que describan de qué trata.
    Texto: \"\"\"{texto_limpio}\"\"\"
    Devuelve solo una lista de palabras o frases cortas separadas por comas.
    """

    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    etiquetas_texto = response.choices[0].message.content.strip()
    # Convertir en lista y limpiar espacios
    etiquetas_finales = [e.strip() for e in etiquetas_texto.split(",")][:max_etiquetas]

    return etiquetas_finales

Lector de documentos

In [67]:
def leer_pdf(contenido, limite_palabras):
    texto = ""
    reader = PdfReader(io.BytesIO(contenido))
    for page in reader.pages:
        if page.extract_text():
            texto += page.extract_text() + "\n"
            if len(texto.split()) >= limite_palabras:
                break
    return texto


def leer_docx(contenido, limite_palabras):
    texto = ""
    doc = docx.Document(io.BytesIO(contenido))
    for p in doc.paragraphs:
        if p.text.strip():
            texto += p.text + "\n"
            if len(texto.split()) >= limite_palabras:
                break
    return texto


def leer_excel(contenido, limite_palabras):
    texto = ""
    wb = openpyxl.load_workbook(io.BytesIO(contenido), data_only=True, read_only=True)
    for sheet in wb.worksheets:
        for row in sheet.iter_rows(values_only=True):
            texto += " ".join([str(cell) for cell in row if cell]) + "\n"
            if len(texto.split()) >= limite_palabras:
                break
        if len(texto.split()) >= limite_palabras:
            break
    return texto


def leer_xls(contenido, limite_palabras):
    texto = ""
    temp_file = "temp.xls"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    wb = xlrd.open_workbook(temp_file)
    for sheet in wb.sheets():
        for row_idx in range(sheet.nrows):
            row = sheet.row_values(row_idx)
            texto += " ".join([str(cell) for cell in row if cell]) + "\n"
            if len(texto.split()) >= limite_palabras:
                break
        if len(texto.split()) >= limite_palabras:
            break
    os.remove(temp_file)
    return texto


def leer_pptx(contenido, limite_palabras):
    texto = ""
    temp_file = "temp.pptx"
    with open(temp_file, "wb") as f:
        f.write(contenido)
    prs = Presentation(temp_file)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text") and shape.text.strip():
                texto += shape.text + "\n"
                if len(texto.split()) >= limite_palabras:
                    break
        if len(texto.split()) >= limite_palabras:
            break
    os.remove(temp_file)
    return texto


def leer_txt_csv(contenido, limite_palabras):
    texto = contenido.decode("utf-8", errors="ignore")
    return " ".join(texto.split()[:limite_palabras])

def leer_archivo(nombre, contenido, limite_palabras=None):
    ext = nombre.split(".")[-1].lower()
    if ext == "pdf":
        return leer_pdf(contenido, limite_palabras)
    elif ext == "docx":
        return leer_docx(contenido, limite_palabras)
    elif ext == "xlsx":
        return leer_excel(contenido, limite_palabras)
    elif ext == "xls":
        return leer_xls(contenido, limite_palabras)
    elif ext == "pptx":
        return leer_pptx(contenido, limite_palabras)
    elif ext in {"txt", "csv"}:
        return leer_txt_csv(contenido, limite_palabras)
    return ""

# Leer archivos de OneDrive

In [70]:
limite_palabras = 500

# Obtener archivos de OneDrive
response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Error al obtener archivos: {response.text}")
data = response.json()

# Filtrar solo archivos con extensiones permitidas
archivos_permitidos = [item for item in data.get("value", [])
                       if any(item["name"].lower().endswith(ext) for ext in ext_permitidas)]

# Tomar solo el primer archivo
if archivos_permitidos:
    item = archivos_permitidos[0]
    nombre = item["name"]
    download_url = item["@microsoft.graph.downloadUrl"]
    file_bytes = requests.get(download_url).content
    texto = leer_archivo(nombre, file_bytes, limite_palabras)
    resultados = {nombre: etiquetar_texto(texto) if texto else []}

    # Mostrar resultado
    for nombre, etiquetas_detectadas in resultados.items():
        print(f"{nombre} → {etiquetas_detectadas}")

    # Guardar en CSV
    import csv
    with open("etiquetas_onedrive.csv", "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["Archivo", "Etiquetas"])
        for nombre, etiquetas_detectadas in resultados.items():
            etiquetas_str = ", ".join(etiquetas_detectadas) if etiquetas_detectadas else "Sin etiquetas"
            writer.writerow([nombre, etiquetas_str])

    print("Resultados guardados en etiquetas_onedrive.csv")
else:
    print("No se encontraron archivos permitidos para procesar.")


11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf → ['registro', 'registro', 'inventario', 'educación', 'temporada', 'justificante', 'freelancer', 'legal', 'arrendamiento', 'anexo', 'vivienda', 'proyecto', 'proyecto', 'contrato', 'alquiler', 'UE', 'empresa', 'administrativo', 'cursos', 'OEPM', 'reserva', 'datos', 'HH-CCSS', 'presentación', 'csv', 'NDA', 'prueba', 'confidencialidad', 'informe', 'prórroga', 'test', 'admitidos', 'unilateral', 'bachillerato', '2011', 'personal', '2012', 'formulario', 'inmobiliaria', 'modelo', '2010', 'inmueble', 'contabilidad', 'opción de compra', 'estados financieros', 'acuerdo', '2016', 'finca rústica', 'plantilla', 'académico', 'biblioteca', 'rescisión', '2013', '2018', '2030', '2014', 'análisis', '2015', '2017', 'monográficos', '2027', '2028', '2029', '2019', '2024', '2023', '2022', '2026', '2025', '2021', '2020']
Resultados guardados en etiquetas_onedrive.csv


#Guardar resultados

In [ ]:
with open("etiquetas_onedrive.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["Archivo", "Etiquetas"])
    for nombre, etiquetas_detectadas in resultados.items():
        # Si no hay etiquetas, poner 'Sin etiquetas'
        etiquetas_str = ", ".join(etiquetas_detectadas) if etiquetas_detectadas else "Sin etiquetas"
        writer.writerow([nombre, etiquetas_str])

print("Resultados guardados en etiquetas_onedrive.csv")


## Codigo para cargar en la etiqueta de descripcion en onedrive

In [ ]:
"""
# ============================
# Configuración
# ============================
json_path = "etiquetas_onedrive.json"
headers = {"Authorization": f"Bearer {access_token}"}

# Carpeta objetivo en OneDrive
carpeta_objetivo = "Etiquetados"

# ============================
# Cargar JSON de etiquetas
# ============================
with open(json_path, "r", encoding="utf-8") as f:
    etiquetas_data = json.load(f)

# ============================
# Aplicar etiquetas en la descripción
# ============================
for archivo, etiquetas in etiquetas_data.items():
    if not etiquetas:
        continue

    # Buscar archivo en OneDrive
    url_file = f"https://graph.microsoft.com/v1.0/me/drive/root:/{carpeta_objetivo}/{archivo}"
    resp_file = requests.get(url_file, headers=headers)

    if resp_file.status_code != 200:
        print(f"⚠️ No se encontró {archivo} en OneDrive")
        continue

    file_id = resp_file.json()["id"]

    # Guardamos etiquetas en el campo "description"
    url_update = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}"
    payload = {"description": ", ".join(etiquetas)}

    response_update = requests.patch(
        url_update,
        headers={**headers, "Content-Type": "application/json"},
        json=payload
    )

    if response_update.status_code in [200, 204]:
        print(f"✅ Etiquetas {etiquetas} aplicadas a {archivo} en la descripción")
    else:
        print(f"⚠️ Error al actualizar {archivo}: {response_update.text}")
"""